#  <font color = blue>Testing the addition to 4 new agents and functions

### <font color = pink> Web Search (bing), Python and Weather Data (historical and current)

## <font color = tomato> ADMIN

In [1]:
import autogen
from typing import Literal
import os

from pydantic import BaseModel, Field
from typing_extensions import Annotated
import requests
import autogen
from autogen.cache import Cache

config_list = [{
    'model': 'gpt-4o',
    'api_key': 'sk-proj-iQtcgUJOOf4n53Bs6uyqT3BlbkFJnEIqUeEwXjbjVMcDVqiz'
}]

llm_config={
    # "request_timeout": 600,
    # "seed": 42,
    "config_list": config_list,
    "temperature": 0
}
os.environ['BING_API_KEY'] = '7e534482ac5f46d3a2a979072b19e591'
bing_api_key = os.environ["BING_API_KEY"]


## <font color = tomato>  DEFINE AGENTS

### <font color = grey> User Proxy

In [2]:
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "group",
        "use_docker": False,},
    max_consecutive_auto_reply=10,
    system_message="""Reply TERMINATE if the task been solved at full satisfaction. 
    Otherwise, reply CONTINUE or the reason why the task is not solved yet. """,

)

### <font color = grey> Current Weather Data Retriever 

In [3]:
current_weather_data_retriever = autogen.AssistantAgent(
    name="current_weather_data_retriever",
    # system_message="For weather forecasting tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
    system_message = """You are the agent specializing in answering all weather related queries.
    When part of a group, please direct all such queries to yourself
    Please only use the functions you have been provided with. Reply TERMINATE when the task is done.""",
    llm_config=llm_config
    
)

### <font color = grey> Historical Weather Data Retriever 

In [4]:
historical_weather_data_retriever = autogen.AssistantAgent(
    name="historical_weather_data_retriever",
    system_message="""You are the agent specializing in retrieving historical weather data.
    When part of a group, please direct all such queries to yourself.
    Please only use the functions you have been provided with. Reply TERMINATE when the task is done.""",
    llm_config=llm_config
)

### <font color = grey> Engineer

In [5]:
engineer = autogen.AssistantAgent(
    name="engineer",
    system_message="""

    For coding tasks, only use the functions you have been provided with. 
    Reply TERMINATE when the task is done.
    
    """,
    llm_config=llm_config,
)

___
## <font color = tomato> DEFINE TOOLS & ASSIGN TO AGENTS

### <font color = grey> Get Current Weather Data 

In [6]:
import os
import requests

@user_proxy.register_for_execution()
@current_weather_data_retriever.register_for_llm(description="function answering all queries related to the weather")
def get_current_weather_data(zipcode: str) -> dict:
    api_key = "864a9bb0b562e1c87e01b38880d5bee7" # tied to personal account
    # api_key = "a5e5edb25b80663446681618c8290ec7" # tied to school account
    # api_key = "4e7828e7712b4d15c2d094441196326f" # tied to school account


    geocoding_url = "http://api.openweathermap.org/geo/1.0/zip"
    one_call_url = "https://api.openweathermap.org/data/3.0/onecall"
    
    # Step 1: Convert zipcode to geographic coordinates
    geocoding_params = {"zip": zipcode, "appid": api_key}
    geo_response = requests.get(geocoding_url, params=geocoding_params)
    if geo_response.status_code != 200:
        return {"error": "Failed to fetch geographic coordinates."}
    
    geo_data = geo_response.json()
    lat, lon = geo_data.get("lat"), geo_data.get("lon")
    
    # Step 2: Fetch weather data using One Call API
    one_call_params = {
        "lat": lat,
        "lon": lon,
        "exclude": "minutely,hourly,alerts",  # Exclude unnecessary data
        "appid": api_key,
        "units": "metric"  # Metric units; change to "imperial" if preferred
    }
    weather_response = requests.get(one_call_url, params=one_call_params)
    if weather_response.status_code != 200:
        return {"error": "Failed to fetch weather data."}
    
    weather_data = weather_response.json()
    current = weather_data.get("current", {})
    daily = weather_data.get("daily", [])

    # Extracting key data
    current_temp = current.get("temp")
    humidity = current.get("humidity")
    wind_speed = current.get("wind_speed")
    weather_description = current.get("weather", [{}])[0].get("description")
    
    daily_temps = [day.get("temp", {}).get("day") for day in daily]
    average_temp = sum(daily_temps) / len(daily_temps) if daily_temps else None

    daily_humidity = [day.get("humidity") for day in daily]
    average_humidity = sum(daily_humidity) / len(daily_humidity) if daily_humidity else None

    daily_wind_speed = [day.get("wind_speed") for day in daily]
    average_wind_speed = sum(daily_wind_speed) / len(daily_wind_speed) if daily_wind_speed else None

    return {
        "current_temperature": current_temp,
        "average_temperature": average_temp,
        "current_humidity": humidity,
        "average_humidity": average_humidity,
        "current_wind_speed": wind_speed,
        "average_wind_speed": average_wind_speed,
        "weather_description": weather_description
    }

### <font color = grey> Get Historical Weather Data 

In [7]:
import os
import requests
import time

def get_historical_weather_data(zipcode: str, start_date: str, end_date: str) -> dict:
    # api_key = "864a9bb0b562e1c87e01b38880d5bee7" # tied to personal account
    api_key = "a5e5edb25b80663446681618c8290ec7" # tied to school account
    geocoding_url = "http://api.openweathermap.org/geo/1.0/zip"
    history_url = "https://history.openweathermap.org/data/2.5/history/city"
    
    try:
        # Convert dates to Unix timestamps
        start_timestamp = int(time.mktime(time.strptime(start_date, "%Y-%m-%d")))
        end_timestamp = int(time.mktime(time.strptime(end_date, "%Y-%m-%d")))

        # Step 1: Convert zipcode to geographic coordinates
        geocoding_params = {"zip": zipcode, "appid": api_key}
        geo_response = requests.get(geocoding_url, params=geocoding_params)
        if geo_response.status_code != 200:
            return {"error": "Failed to fetch geographic coordinates."}

        geo_data = geo_response.json()
        lat, lon = geo_data.get("lat"), geo_data.get("lon")
        if lat is None or lon is None:
            return {"error": "Invalid geographic coordinates."}

        # Step 2: Fetch historical weather data using History API
        history_params = {
            "lat": lat,
            "lon": lon,
            "type": "hour",
            "start": start_timestamp,
            "end": end_timestamp,
            "appid": api_key,
            "units": "metric"  # Metric units; change to "imperial" if preferred
        }
        history_response = requests.get(history_url, params=history_params)
        
        # Debugging information
        print("Request URL:", history_response.url)
        print("Status Code:", history_response.status_code)
        print("Response Text:", history_response.text)
        
        if history_response.status_code != 200:
            return {"error": "Failed to fetch historical weather data."}

        history_data = history_response.json()
        return history_data
    
    except Exception as e:
        return {"error": str(e)}

# Example usage
zipcode = "95321"  # Zip code for Groveland, CA
start_date = "2022-01-01"
end_date = "2022-01-10"

weather_data = get_historical_weather_data(zipcode, start_date, end_date)
print(weather_data)

Request URL: https://history.openweathermap.org/data/2.5/history/city?lat=37.8298&lon=-120.1037&type=hour&start=1640995200&end=1641772800&appid=a5e5edb25b80663446681618c8290ec7&units=metric
Status Code: 401
Response Text: {"cod":401, "message": "Invalid API key. Please see http://openweathermap.org/faq#error401 for more info."}
{'error': 'Failed to fetch historical weather data.'}


### <font color = grey> Execute Python Code

In [8]:
# one way of registering functions is to use the register_for_llm and register_for_execution decorators
@user_proxy.register_for_execution()
@engineer.register_for_llm(description="run cell in ipython and return the execution result.")
def exec_python(cell: Annotated[str, "Valid Python cell to execute."]) -> str:
    ipython = get_ipython()
    result = ipython.run_cell(cell)
    log = str(result.result)
    if result.error_before_exec is not None:
        log += f"\n{result.error_before_exec}"
    if result.error_in_exec is not None:
        log += f"\n{result.error_in_exec}"
    return log

### <font color = grey> Web Surfer with Bing

In [9]:
from autogen.agentchat.contrib.web_surfer import WebSurferAgent  # noqa: E402

web_surfer = WebSurferAgent(
    "web_surfer",
    llm_config=llm_config,
    summarizer_llm_config=llm_config,
    browser_config={"viewport_size": 4096, "bing_api_key": bing_api_key},
)

___
#### <font color = tomato> Check Agent and Tool Assignments

In [10]:
current_weather_data_retriever.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'function answering all queries related to the weather',
   'name': 'get_current_weather_data',
   'parameters': {'type': 'object',
    'properties': {'zipcode': {'type': 'string', 'description': 'zipcode'}},
    'required': ['zipcode']}}}]

In [13]:
# historical_weather_data_retriever.llm_config["tools"]

In [14]:
engineer.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'run cell in ipython and return the execution result.',
   'name': 'exec_python',
   'parameters': {'type': 'object',
    'properties': {'cell': {'type': 'string',
      'description': 'Valid Python cell to execute.'}},
    'required': ['cell']}}}]

## <font color = tomato> TEST AGENTS & FUNCTIONS

In [15]:
# Example usage
zipcode = "95321"  # San Francisco, Zip Code 94131
weather_data = get_current_weather_data(zipcode)
print(weather_data)

{'current_temperature': 23.16, 'average_temperature': 35.06875, 'current_humidity': 32, 'average_humidity': 15.125, 'current_wind_speed': 1.92, 'average_wind_speed': 4.6925, 'weather_description': 'clear sky'}


In [16]:
# Example usage
zipcode = "95321"  # Zip code for Groveland, CA
start_date = "2022-01-01"
end_date = "2022-01-10"

weather_data = get_historical_weather_data(zipcode, start_date, end_date)
print(weather_data)

Request URL: https://history.openweathermap.org/data/2.5/history/city?lat=37.8298&lon=-120.1037&type=hour&start=1640995200&end=1641772800&appid=a5e5edb25b80663446681618c8290ec7&units=metric
Status Code: 401
Response Text: {"cod":401, "message": "Invalid API key. Please see http://openweathermap.org/faq#error401 for more info."}
{'error': 'Failed to fetch historical weather data.'}


---
# <font color = blue>  Run Python & Web Search

In [ ]:
# task = "Can you get the current weather information for San Francisco, Zip Code 94550?, \
#         and plot the returned values in a table using the python tool available to you\
#         please answer this question only using the approrpriate tool when required\
#         for example, please use the weather related tool for all weather related queries\
#         please use the search tool only if specifically asked to search the web for answer"

In [19]:
task = "Can you get the historical weather information for Livermore, CA, Zip Code 94550?, \
        for the period 01-Nov-2005 to 10-Nov-2005 and then plot the returned values in a bar chart \
        using the python tool available to you\
        please answer this question only using the approrpriate tool when required\
        for example, please use the historical weather related tool and current weather tool differently\
        as the case may be\
        please use the search tool only if specifically asked to search the web for answer"

In [21]:
# groupchat = autogen.GroupChat(agents=[user_proxy, current_weather_data_retriever, historical_weather_data_retriever, \
#                                       engineer, web_surfer], messages=[], max_round=5)

# manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [22]:
groupchat = autogen.GroupChat(agents=[user_proxy, current_weather_data_retriever, \
                                      engineer, web_surfer], messages=[], max_round=5)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [23]:
with Cache.disk() as cache:
    # start the conversation
    res = user_proxy.initiate_chat(
        manager, message=task, summary_method="reflection_with_llm", cache=cache, llm_config=llm_config
    )

user_proxy (to chat_manager):

Can you get the historical weather information for Livermore, CA, Zip Code 94550?,         for the period 01-Nov-2005 to 10-Nov-2005 and then plot the returned values in a bar chart         using the python tool available to you        please answer this question only using the approrpriate tool when required        for example, please use the historical weather related tool and current weather tool differently        as the case may be        please use the search tool only if specifically asked to search the web for answer

--------------------------------------------------------------------------------

Next speaker: current_weather_data_retriever

current_weather_data_retriever (to chat_manager):

I can only provide current weather data using the tools available to me. Unfortunately, I do not have access to historical weather data or the ability to plot charts. If you need the current weather information for Livermore, CA, Zip Code 94550, I can provid

In [ ]:
# with Cache.disk() as cache:
#     # start the conversation
#     user_proxy.initiate_chat(
#         engineer,
#         # message=task,
#         message="""
        
#         Can you plot the average temperature in San Francisco, Zip Code 94131 over the past 30 days?
        
#         """,
#         cache=cache,
#     )